In [2]:
import mysql.connector
import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt
from streamlit_option_menu import option_menu
import numpy as np

2024-12-08 01:10:31.335 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


**OVERRIDE REPORT**

In [31]:
#importing data
override =   pd.read_excel("Override Sales Report.xlsx")
override.columns = override.columns.str.lower().str.strip()
#PREPROCESSING
#for store id column
override = override[~override['store name'].str.contains("Market:",na=False)]
override['store id'] = override['store name'].where(override['store name'].str.contains("StoreID:", na=False))
override['store id'] = override['store id'].fillna(method='ffill')
override['store id'] = override['store id'].str.replace("StoreID: ", "", regex=False).str.strip()
override = override.drop(columns=['store name'])  # Remove original column
#for item# column
condition_null = override['item #'].isnull()
condition_in_list = override['item #'].isin(["ACT/QPAY DISCOUNT", "NC128TRIPLESIM"])
override = override[~(condition_null | condition_in_list)]
#for serial no. column
override = override[override['serial no.'].isnull()]
#for discount column
override['discount'] = override['min price'] - override['price']
override = override[override['discount'] >= 0]

C:\Users\Desktop\AppData\Local\Temp\ipykernel_10124\1024990883.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  override['store id'] = override['store id'].fillna(method='ffill')


**MAIN DISCOUNT REPORT**

In [32]:
#importing data
disc_report = pd.read_excel("map.xlsx")
disc_report.rename(columns={"Store ID": "Store_ID"}, inplace=True)
#PREPROCESSING
#for limit column
if "limit" not in disc_report.columns:
    lst = ["NORTHWEST HWY", "704 JEFFERSON", "DUNCANVILLE", "COLORADO BLVD", "JACKSBORO"]
    disc_report["Store_Limit"] = np.where(
        disc_report["Store"].isin(lst),  # Condition: if 'Store' is in the list
        1500,                           # Value if condition is True
        250                             # Value if condition is False
    )
#for Override Disc
override.rename(columns={"store id": "Store_ID"}, inplace=True)
override_summed = override.groupby("Store_ID", as_index=False)["discount"].sum()
disc_report = disc_report.merge(override_summed, how="left", left_on="Store_ID", right_on="Store_ID")
disc_report.rename(columns={"discount": "Override_Disc"}, inplace=True)
disc_report["Override_Disc"] = disc_report["Override_Disc"].fillna(0)
#for Disc SKU
salesbycategory = pd.read_excel("SalesbyCategory.xls")
salesbycategory.rename(columns={"custno": "Store_ID"}, inplace=True)
salesbycategory_summed = salesbycategory.groupby("Store_ID", as_index=False)["price"].sum()
disc_report = disc_report.merge(salesbycategory_summed, how="left", left_on="Store_ID", right_on="Store_ID")
disc_report.rename(columns={"price": "Disc_SKU"}, inplace=True)
disc_report["Disc_SKU"] = disc_report["Disc_SKU"].fillna(0)
#for remaining cols
required_columns = ['Market', 'Store', 'Store_ID', 'Store_Limit', 'Override_Disc',
                    'Disc_SKU', 'Total_Availed', 'Remaining', 'EOL', 'Aging',
                    'Cx_Survey', 'MD_approved', 'Comment']
for column in required_columns:
    if column not in disc_report.columns:
        disc_report[column] = None

WARNING *** file size (378893) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


In [40]:
disc_report.columns

Index(['Market', 'Store', 'Store_ID', 'Store_Limit', 'Override_Disc',
       'Disc_SKU', 'Total_Availed', 'Remaining', 'EOL', 'Aging', 'Cx_Survey',
       'MD_approved', 'Comment'],
      dtype='object')

**DATABASE PROCESSING**

In [37]:
# Establish a connection to MySQL Server
mydb = mysql.connector.connect(
    host='Localhost',
    user='root',
    password = '34&*^&AsXti2098as3#$><?',
    database='rtpos')

mycursor = mydb.cursor()

# Function to update database with DataFrame
def update_or_insert_database(dataframe, connection, cursor):
    # Iterate through the rows of the dataframe
    for index, row in dataframe.iterrows():
        # Query to check if Store_ID exists
        check_query = "SELECT 1 FROM desc_report WHERE Store_ID = %s"
        cursor.execute(check_query, (row['Store_ID'],))
        result = cursor.fetchone()

        if result:  # If Store_ID exists, update the record
            update_query = """
                UPDATE desc_report
                SET 
                    Market = %s,
                    Store = %s,
                    Store_Limit = %s,
                    Override_Disc = %s,
                    Disc_SKU = %s,
                    EOL = %s,
                    Aging = %s,
                    Cx_Survey = %s,
                    MD_approved = %s,
                    Comment = %s
                WHERE Store_ID = %s
            """
            data_tuple = (
                row['Market'], row['Store'], row['Store_Limit'], row['Override_Disc'], row['Disc_SKU'], 
                row['EOL'], row['Aging'], row['Cx_Survey'], row['MD_approved'], row['Comment'], row['Store_ID']
            )
            cursor.execute(update_query, data_tuple)
        else:  # If Store_ID does not exist, insert the record
            insert_query = """
                INSERT INTO desc_report (
                    Market, Store, Store_ID, Store_Limit, Override_Disc, Disc_SKU, 
                    Total_Availed, Remaining, EOL, Aging, Cx_Survey, MD_approved, Comment
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            data_tuple = (
                row['Market'], row['Store'], row['Store_ID'], row['Store_Limit'], 
                row['Override_Disc'], row['Disc_SKU'], row['Total_Availed'], row['Remaining'],
                row['EOL'], row['Aging'], row['Cx_Survey'], row['MD_approved'], row['Comment']
            )
            cursor.execute(insert_query, data_tuple)
    
    # Commit the changes and close the connection
    connection.commit()
    mycursor.close()
    mydb.close()


In [38]:
update_or_insert_database(disc_report, mydb, mycursor)